# Lab 7 – K-Means avec données réelles (Cust_Segmentation.csv)

Ce notebook : 
1) charge le CSV réel, 
2) nettoie et standardise, 
3) **explore k** (Elbow + Silhouette sur un échantillon pour vitesse), 
4) choisit k automatiquement (max silhouette), 
5) entraîne KMeans sur **toutes** les données, 
6) ajoute `Cluster` et affiche un résumé par cluster, 
7) trace une PCA 2D.

Assure-toi que **`Cust_Segmentation.csv`** est **à côté du notebook**.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

csv_filename = "Cust_Segmentation.csv"  # placer ce fichier à côté du notebook
cust_df = pd.read_csv(csv_filename)

# Drop éventuel et numériques
df = cust_df.copy()
if "Address" in df.columns:
    df = df.drop(columns=["Address"])
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

X_raw = df[num_cols].values
X_raw = np.nan_to_num(X_raw, nan=0.0, posinf=1e6, neginf=-1e6)
X = StandardScaler().fit_transform(X_raw)

# --- Exploration rapide de k (échantillon + MiniBatchKMeans) ---
n = X.shape[0]
sample_size = min(2000, n)
rng = np.random.default_rng(42)
idx = rng.choice(n, size=sample_size, replace=False) if sample_size < n else np.arange(n)
X_s = X[idx]

K_range = [2,3,4,5,6,7,8]
inertias, silhouettes = [], []
for k in K_range:
    km = MiniBatchKMeans(n_clusters=k, random_state=42, batch_size=256, n_init=3, max_no_improvement=5)
    labels_k = km.fit_predict(X_s)
    inertias.append(km.inertia_)
    # silhouette sur sous-ensemble si nécessaire (<=2000 déjà)
    silhouettes.append(silhouette_score(X_s, labels_k))

plt.figure(); plt.plot(K_range, inertias, marker="o"); plt.title("Elbow (MiniBatch, sample)"); plt.xlabel("k"); plt.ylabel("WCSS"); plt.grid(True); plt.show()
plt.figure(); plt.plot(K_range, silhouettes, marker="o"); plt.title("Silhouette (MiniBatch, sample)"); plt.xlabel("k"); plt.ylabel("Silhouette"); plt.grid(True); plt.show()

best_k = int(K_range[int(np.argmax(silhouettes))])
print("Chosen k (by silhouette on sample):", best_k)

# --- Ajustement final sur toutes les données ---
kmeans_final = KMeans(n_clusters=best_k, init="k-means++", n_init=5, random_state=42)
labels_final = kmeans_final.fit_predict(X)

df_with_clusters = cust_df.copy()
df_with_clusters["Cluster"] = labels_final
display(df_with_clusters.head())

# Résumé par cluster
cluster_summary = df_with_clusters.groupby("Cluster")[num_cols].mean().round(2)
display(cluster_summary)

# PCA (échantillon pour vitesse d'affichage)
pca = PCA(n_components=2, random_state=42)
XY = pca.fit_transform(X_s)
labels_s = kmeans_final.predict(X_s)
plt.figure(); plt.scatter(XY[:,0], XY[:,1], c=labels_s, alpha=0.7); plt.title(f"K-Means (k={best_k}) – PCA (sample)"); plt.xlabel("PC1"); plt.ylabel("PC2"); plt.grid(True); plt.show()
